# Introduction to Sherpany JupyterHub & Notebooks

## 1. Structure & General Stuff
The <a href='https://wiki.sherpany.com/display/EN/JupyterHub'>JupyterHub wiki</a> contains documentation of our libraries and setup.

The *utils* folder contains various tools and libraries which facilitate the data access and processing.
*Setup & Models* is still being built up and will contain notebooks containing theory and implementation examples for various predictive and statistical methods.

All projects (datascience projects or general insight reporting) are in the *Projects* folder. As documented on the wiki, each folder follows a defined structure with the ticket number at the beginning.

### 1.1. Execution of Code Cells
There are multiple ways to run code in a cell.
- shift-enter: runs cell and selects next one below
- ctrl-enter: runs cell and selects current cell
- alt/option-enter: runs cell and inserts a new one below

## 2. Markdown
All text elements in reports are done in markdown cells such as this one. It uses the standard <a href='https://daringfireball.net/projects/markdown/' > Mardown </a> format.
### 2.1. Titles
One of the most used elements are titles. They can be created by adding # in front of the title text.
The number of # determines the order of title with one # corresponding to main headers.

Execute this cell to show examples:
# Example Title 1
## Example Title 1.1
### Example Title 1.1.1
#### Example Title 1.1.1.1.
##### Example Title 1.1.1.1.1.

### 2.2. General Formatting
**Bold text** is done with ** before and after the text.

*Italic text* is done with * before and after the text.

horizontal line with ---

---

### 2.3. Lists
Bullet points are easily done by starting each line with -, whereas numbered lists can be created by adding the respective numbers in front.

Execute this cell with shift-enter to show examples:
- bullet point list
- second item

1. numbered list
2. second item

### 2.4. Table
A simple table can be created as demonstrated below:

| This | is   |
|------|------|
|   a  | table|

## 3. Data Examples
This segment gives a short introduction in simple sql queries and how to do them within JupyterHub.
### 3.1. Load Libraries
As mentioned above, the *utils* folder contains all functions necessary to query the backend postgreSQL db.
Connection and extraction is done with the sql function of the common module and the <a href = 'https://wiki.sherpany.com/display/EN/SQL+Constants'>sql_constants</a> helps writing short and easy to read queries.

In [10]:
import sys
sys.path.append('/opt/notebooks/utils');
from common import sql
import sql_constants as c

### 3.2. Search DataBase Tables
As there are 204 different tables in our database. Finding the right table can sometimes be tricky. The following command lists all tables which title contains 'comment'.

In [11]:
sql("select TABLE_NAME from information_schema.tables where table_name LIKE '%comment%'")

[('comments_comment',),
 ('comments_commentstatus',),
 ('comments_mention',),
 ('permissions_commentthreadmemberaccess',),
 ('comments_commentthread',)]

The next step is to look at the columns in a table and what type of data they contain.

In [12]:
sql("select column_name,data_type from information_schema.columns where table_name = 'comments_commentthread'")

[('created', 'timestamp with time zone'),
 ('modified', 'timestamp with time zone'),
 ('deleted', 'timestamp with time zone'),
 ('id', 'uuid'),
 ('state', 'text'),
 ('meeting_document_page', 'smallint'),
 ('agenda_item_id', 'integer'),
 ('event_id', 'integer'),
 ('meeting_document_id', 'integer')]

### 3.3. Examples
#### Example 1: count
Extract the total number of active demo rooms.

In [13]:
sql("SELECT count(DISTINCT r.id) "
    "FROM rooms_room r "
    "WHERE r.is_active AND r.is_demo")

[(379,)]

#### Example 2: count & aggregate
Extract a table with the count of meetings per productive room id. For this, we group all meeting ids by room ids. *sql_constants* contains a query which already filters for all room ids which are active (*is_active*), not demo (*NOT is_demo*) and not sherpany (*NOT is_sherpany*). Extracting an entire dataframe can be done using the *queryToDF* function which returns a pandas dataframe.

In [9]:
from common import queryToDF
query = ("SELECT DISTINCT count(DISTINCT m.id), m.room_id "
         "FROM meetings_meeting m "
         "WHERE m.id IN ("+c.EVENTS_IN_PRODUCTIVE_ROOM_IDS+")"
         "GROUP BY m.room_id ")
data = queryToDF(query)
data.head()

,count(m.id),m.room_id
0,26,839
1,379,471
2,1,1446
3,10,1766
4,26,662


#### Example 3: csv export
If data needed in a standard format such as a csv, the common module includes *query_with_automated_custom_headers_to_csv* which writes a csv file with the data extracted using the query. Here we take the same query as above, but export it automatically via csv.

In [8]:
from common import query_with_automated_custom_headers_to_csv

query_with_automated_custom_headers_to_csv(query, 'file_name.csv')

#### Example 4: members of a certain room
Here we're filtering for a specific room, of which we don't know the exact name or id and count the total attendees of meetings in the last 3 months.

In [3]:
sql("SELECT name, id FROM rooms_room WHERE name LIKE '%Handelskammer%'")

[('Handelskammer beider Basel', 749)]

In [ ]:
in_meetings = sql("SELECT count(DISTINCT n.id) "
    "FROM members_meetingmember mm "
    "JOIN members_member n ON mm.member_id = n.id "
    "JOIN meetings_meeting m ON mm.meeting_id = m.id "
    "WHERE n.room_id = 749 "
    "AND m.start > NOW() - INTERVAL '6 months' "
    "AND mm.is_attendee ")[0][0]
in_meetings

Now we want to see the proportion of the total members. According to <a href='https://wiki.sherpany.com/display/EN/Data+Models'>our wiki page for models</a>, role has to equal 1 for members. 

In [15]:
total = sql("SELECT count(DISTINCT n.id) "
    "FROM members_member n "
    "WHERE n.room_id = 749 "
    "AND n.role = 1 "
    "AND n.id IN ("+c.ACTIVE_AND_DRAFT_MEMBER_IDS+") "
    "AND n.user_id IN ("+c.NON_SHERPANY_USER_IDS+")")[0][0]
total

34

In [17]:
round(in_meetings / total * 100, 2)

100.0

Appearently, all members in this specific room attended at least 1 meeting in the last 3 months.